# Westeros Tutorial
## Adding share constraints

In this tutorial we will look at how to add *share_constraints* to the model. These constraints are aimed at limiting the activity of a single/set of technologies to a certain share of the activity of another single/set of technologies. This is a very useful feature for policy analysis. In many of the nationaly determined contributions (NDCs) as part of the Paris Agreement (*UNFCCC. FCCC/CP/2015/L.9/Rev.1: Adoption of the Paris Agreement UNFCCC (2015)*), numerous countries formulated part of their targets in terms of *shares*. The European Union for example, has comitted to a target, where renewables will make up 30% of the energy mix by 2030. How to implement such a policy in an energy model will be the focus of this tutorial.

## Adding a minimum share constraint
Building on the '*baseline*' scenario, we will add a share constraint in which we assume that Westeros has set itself a target to generate 50% of its electricity from wind power by 720.

In [ ]:
import pandas as pd
import ixmp as ix
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ix.Platform(dbtype='HSQLDB')

## Load the existing scenario '*baseline*' and clone to a new scenario '*share_constraint*' for which we will introduce a share constraint.

In [ ]:
model = 'Westeros Electrified'
base = message_ix.Scenario(mp, model=model, scenario='baseline')
scen = base.clone(model, 'share_constraint', 'illustration of share-constraint formulation', keep_solution=False)
scen.check_out()

## Retrieve parameters to perform subsequent addition of parameters

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Steps required to add share constraints
The following steps are required in order to introduce a share constraint, as demonstrated below:
1. Define a new 'share' and add this to the *set* `shares`
2. Define which technologies contribute towards the total
3. Define which technologies contribute towards fullfilling the share
4. Define the share for relevant timesteps

The implementation is generic and flexible, so that any combination of commodities, levels, technologies and nodes can be put in relation to any other combination.

### Step1: Define a new 'share' and add this to the *set* `shares`
We will define a new share called `renewable_share_of_electricity`

In [ ]:
shares = 'renewable_share_of_electricity'
scen.add_set('shares', shares)

### Step2: Define which technologies make up the group accounting for the total, to which the share will be applied. 
### Set corresponding commodity and level for which the share constraint applies.
The aim is to increase the share of electricity generated by renewables to 50% in 720. Therefore, we will need to define, which technologies contribute to total electricity generation at the secondary energy level.  `wind_ppl` and `coal_ppl` will be added to the `type_tec`: `share_total`.  For this `type_tec` we need to the relevant node, mode, level and commodity which will be accounted for, and add this to the *set* `map_shares_commodity_total`.

In [ ]:
type_tec = 'share_total'
scen.add_cat('technology', type_tec, 'wind_ppl')
scen.add_cat('technology', type_tec, 'coal_ppl')
# Index for 'map_shares_commodity_share' is ['shares', 'node_share', 'node',
#                                            'type_tec', 'mode', 'commodity',
#                                            'level']
df = pd.DataFrame({'shares': [shares],
                   'node_share': country,
                   'node': country,
                   'type_tec': type_tec,
                   'mode': 'standard',
                   'commodity': 'electricity',
                   'level': 'secondary',
})
scen.add_set('map_shares_commodity_total', df)

### Step3: Define which technologies contribute towards fullfilling the share
### Set corresponding commodity and level for which the share constraint applies.
Similar to what was done in the previous step, we will now continue to define which technology will need to provide the 50% renewable energy; in our example this will be the `wind_ppl`

In [ ]:
type_tec = 'share_renewable'
scen.add_cat('technology', type_tec, 'wind_ppl')
# Index for 'map_shares_commodity_share' is ['shares', 'node_share', 'node',
#                                            'type_tec', 'mode', 'commodity',
#                                            'level']
df = pd.DataFrame({'shares': [shares],
                   'node_share': country,
                   'node': country,
                   'type_tec': type_tec,
                   'mode': 'standard',
                   'commodity': 'electricity',
                   'level': 'secondary',
})
scen.add_set('map_shares_commodity_share', df)

### Step4: Define the actual share for relevant timesteps
Lastly, the actual share value needs to be added to the model. As we want to achieve that renewables account for a minimum of 50% of total electricity generation, we will add a parameter `share_commodity_lo`. If in the case that this constraint should act as an upper bound, then the parameter `share_commodity_up` can be used.

In [ ]:
# Index for 'share_commodity_lo' is ['shares', 'node_share', 'year_act',
#                                    'time', 'value', 'unit']
df = pd.DataFrame({'shares': shares,
                   'node_share': country,
                   'year_act': [720],
                   'time': 'year',
                   'value': [.5],
                   'unit': '%'})
scen.add_par('share_commodity_lo', df)

### Commit and solve

In [ ]:
scen.commit(comment='define parameters for minimum renewable share constraint')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

# Plotting Results

In [ ]:
from tools import Plots
p = Plots(scen, country, firstyear=700) # scenario: 'share_constraint' (baseline with share constraint)
b = Plots(base, country, firstyear=700) # scenario: 'baseline' (without share constraint)

## Activity
***
In the new scenario ('*share_constraint*'), the activity of the `wind_ppl` now accounts for 50% of total electricity generation in 720. The activity in 710 of the `wind_ppl` is also slightly higher than in the '*baseline*', caused by the fact that the `wind_ppl` has a diffusion constraint, limiting the activity growth rate, and in order to achieve the activity level required in 720, additional activity is also required in the previous timestep.

### Scenario: '*baseline*'

In [ ]:
b.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Scenario: '*share_constraint*'

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

## Capacity
***
There is big increase in 720 regarding activity which is reflected in the capacity for the two technologies.

### Scenario: '*baseline*'

In [ ]:
b.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Scenario: '*share_constraint*'

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

## Prices
***
As expected, electricity prices in 720 have also increased vis-a-vis the '*baseline*'

### Scenario: '*baseline*'

In [ ]:
b.plot_prices(subset=['light'], baseyear=True)

### Scenario: '*share_constraint*'

In [ ]:
p.plot_prices(subset=['light'], baseyear=True)

In [ ]:
mp.close_db()